In [1]:
import sys
sys.path.append("../src/")

In [2]:
from utils import CVsample, talib, lgb_forcast
from utils import *

In [3]:
df = pd.read_csv("../data/raw_data/train.csv")

In [4]:
weight = pd.read_csv("../data/raw_data/asset_details.csv")

In [5]:
ws = []

### Lightgbm

In [6]:
lgb_params = {'boost': 'gbdt'
         ,'num_leaves': 11
         ,'n_estimators':4000
         ,'max_depth': 11
         ,'subsample_for_bin':200000
         ,'learning_rate':0.05
         ,'min_child_samples':20
         ,'n_jobs':-1
         }


def main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight):
    wPCC_asset_list = {}
    for Asset_ID in range(0, 14):
        dft = df[df['Asset_ID'] == Asset_ID].sort_values(by=['timestamp'], ascending=True)
        # sample
        cvs = CVsample(df=dft, ival='timestamp', cv=cv, plength=plength, ptrain=ptrain, pgap=pgap)
        cvs.main()

        # test
        test = dft[dft['timestamp'] > max(cvs.validation_ends)]
        tb = talib(test)
        test_panel = tb.main()

        # 对每个fold, 有多种方法，平均还是最好
        lgbs, wPCC = [], []
        for i in range(0, cv):
            [train, gap, validation] = cvs.get_df(i)

            # msing data
            ms = MsData(train, timestamp_fill=timestamp_fill, method=method, target='Target')
            train = ms.main()

            # feature
            tb = talib(train)
            train_panel = tb.main()

            tb = talib(validation)
            val_panel = tb.main()

            # lightgbm
            lgb = lgb_forcast(train_panel, val_panel, lgb_params)
            lgb.lgb_train()
            lgbs.append(lgb)
            predict_sigma = math.sqrt(lgb.out_params['cov'][0][0])
            real_sigma = math.sqrt(lgb.out_params['cov'][1][1])
            wPCC_i = weight[weight['Asset_ID'] == Asset_ID]['Weight'].iloc[0]*lgb.out_params['cov'][0][1] / (predict_sigma*real_sigma)
            wPCC.append(wPCC_i)
        feacols = [col for col in train_panel.columns if col not in ['Target', 'timestamp']]
        if cv_method == 'best':
            test['pred'] = lgbs[wPCC.index(max(wPCC))].lgr.predict(test_panel[feacols])
        if cv_method == 'avg':
            test['pred'] = lgbs[0].lgr.predict(test_panel[feacols])
            for i in range(1, cv):
                test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])
            test['pred'] = test['pred'] / cv

        # calculate the final wPCC
        test2 = test[(test['Target'].isnull() == False) & (test['pred'].isnull() == False)]
        cov = np.cov(test2['pred'], test2['Target'])
        wPCC_asset = weight[weight['Asset_ID'] == Asset_ID]['Weight'].values[0]*cov[0][1]/(math.sqrt(cov[0][0]*cov[1][1]))
        wPCC_asset_list[Asset_ID] = wPCC_asset
    return wPCC_asset_list

In [7]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'best'
# missing data
timestamp_fill = False
method = 'drop'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1234155, number of used features: 56
[LightGBM] [Info] Start training from score 0.000018
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049006 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1230466, number of used features: 56
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


...

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058537 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1198986, number of used features: 56
[LightGBM] [Info] Start training from score 0.000012
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1198986, number of used features: 56
[LightGBM] [Info] Start training from score 0.000010
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.09620184390181802, 1: 0.07421432838986834, 2: -0.022736489828096573, 3: 0.08900626786309172, 4: -0.032479740518214596, 5: -0.01638057137218082, 6: 0.007126843025351401, 7: -

/tmp/ipykernel_353353/2554929782.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[wPCC.index(max(wPCC))].lgr.predict(test_panel[feacols])


In [8]:
print(wPCC_asset_list)

{0: 0.09620184390181802, 1: 0.07421432838986834, 2: -0.022736489828096573, 3: 0.08900626786309172, 4: -0.032479740518214596, 5: -0.01638057137218082, 6: 0.007126843025351401, 7: -0.009643009292618939, 8: 0.02706552708416497, 9: 0.005636958868704742, 10: -0.00895741222431711, 11: 0.0019899808803909374, 12: -0.00018311053213950192, 13: -0.02572267255006376}


In [9]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'best'
# missing data
timestamp_fill = False
method = 'linear'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060166 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1243276, number of used features: 56
[LightGBM] [Info] Start training from score 0.000018
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1243276, number of used features: 56
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

...

/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030202 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1199718, number of used features: 56
[LightGBM] [Info] Start training from score 0.000012
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1199718, number of used features: 56
[LightGBM] [Info] Start training from score 0.000010
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.05246783605879326, 1: 0.0065545432996967955, 2: 0.0007675356055733198, 3: 0.0666621743367279, 4: -0.03987825520331999, 5: -0.013040664187379312, 6: 0.014538169495402225, 7: 

/tmp/ipykernel_353353/2554929782.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[wPCC.index(max(wPCC))].lgr.predict(test_panel[feacols])


In [10]:
print(wPCC_asset_list)

{0: 0.05246783605879326, 1: 0.0065545432996967955, 2: 0.0007675356055733198, 3: 0.0666621743367279, 4: -0.03987825520331999, 5: -0.013040664187379312, 6: 0.014538169495402225, 7: 0.0030907956883619104, 8: 0.026245995796984316, 9: 0.005703153301006097, 10: -0.004039099904232205, 11: -0.009771756984453806, 12: -0.0006027205503427688, 13: -0.02637537380880016}


In [11]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'best'
# missing data
timestamp_fill = True
method = 'drop'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032711 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1234155, number of used features: 56
[LightGBM] [Info] Start training from score 0.000018
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1230466, number of used features: 56
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


...

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057917 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1198986, number of used features: 56
[LightGBM] [Info] Start training from score 0.000012
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1198986, number of used features: 56
[LightGBM] [Info] Start training from score 0.000010
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.09578043773018241, 1: 0.07412014797754697, 2: -0.022926784700836145, 3: 0.0888494837805019, 4: -0.032477749829387344, 5: -0.016397379445335685, 6: 0.007122446690736907, 7: -

/tmp/ipykernel_353353/2554929782.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[wPCC.index(max(wPCC))].lgr.predict(test_panel[feacols])


In [12]:
print(wPCC_asset_list)

{0: 0.09578043773018241, 1: 0.07412014797754697, 2: -0.022926784700836145, 3: 0.0888494837805019, 4: -0.032477749829387344, 5: -0.016397379445335685, 6: 0.007122446690736907, 7: -0.008901707087019234, 8: 0.027231776038190297, 9: 0.005680161899039532, 10: -0.008965269566132769, 11: 0.0020207260312441445, 12: -0.00018192508061342334, 13: -0.025748271724779698}


In [13]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'best'
# missing data
timestamp_fill = True
method = 'linear'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010084 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6483
[LightGBM] [Info] Number of data points in the train set: 1251925, number of used features: 50
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6713
[LightGBM] [Info] Number of data points in the train set: 1256790, number of used features: 56
[LightGBM] [Info] Start training from score -0.000006
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


...

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030548 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14271
[LightGBM] [Info] Number of data points in the train set: 1200750, number of used features: 56
[LightGBM] [Info] Start training from score 0.000012
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028682 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14077
[LightGBM] [Info] Number of data points in the train set: 1200750, number of used features: 56
[LightGBM] [Info] Start training from score 0.000010
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.01258199248994704, 1: 0.06941913452327604, 2: 0.011816573063974968, 3: -0.0024471790456871093, 4: 0.019180612

/tmp/ipykernel_353353/2554929782.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[wPCC.index(max(wPCC))].lgr.predict(test_panel[feacols])


In [14]:
print(wPCC_asset_list)

{0: 0.01258199248994704, 1: 0.06941913452327604, 2: 0.011816573063974968, 3: -0.0024471790456871093, 4: 0.01918061261324378, 5: -0.009926177110107596, 6: -0.007171317034889778, 7: 0.028874708513107212, 8: 0.021402111639919504, 9: 0.012176403645110278, 10: -0.002668379160302527, 11: 0.004526453301219037, 12: -0.006583322482007493, 13: 0.015898458560523725}


In [15]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'avg'
# missing data
timestamp_fill = False
method = 'drop'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030653 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1234155, number of used features: 56
[LightGBM] [Info] Start training from score 0.000018
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030021 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1230466, number of used features: 56
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


...

/tmp/ipykernel_353353/2554929782.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[0].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.03847830857395089, 1: 0.04666326377325679, 2: -0.01467523750814525, 3: 0.09906207621244092, 4: -0.04335166718751135, 5: -0.008624017575904714, 6: -0.0004786438833272888, 7: 0.00016776150195685908, 8: 0.029095918492397337, 9: 0.0120891637532644, 10: -0.005665905836979982, 11: -0.0018592588259744138, 12: 0.0044596474174430495, 13: -0.029516634392261355}
CPU times: user 2d 5h 18min 8s, sys: 58min 29s, total: 2d 6h 16min 37s
Wall time: 37min 5s


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])
/tmp/ipykernel_353353/2554929782.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred'] / cv


In [16]:
print(wPCC_asset_list)

{0: 0.03847830857395089, 1: 0.04666326377325679, 2: -0.01467523750814525, 3: 0.09906207621244092, 4: -0.04335166718751135, 5: -0.008624017575904714, 6: -0.0004786438833272888, 7: 0.00016776150195685908, 8: 0.029095918492397337, 9: 0.0120891637532644, 10: -0.005665905836979982, 11: -0.0018592588259744138, 12: 0.0044596474174430495, 13: -0.029516634392261355}


In [17]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'avg'
# missing data
timestamp_fill = False
method = 'linear'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059572 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1243276, number of used features: 56
[LightGBM] [Info] Start training from score 0.000018
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1243276, number of used features: 56
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.target].interpolate(method='linear', inplace=True)
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

...

/tmp/ipykernel_353353/2554929782.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[0].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.03010742017410797, 1: 0.011425885666727932, 2: -0.016368366672512798, 3: 0.09981973136869705, 4: -0.04166187931922221, 5: -0.010135255686296042, 6: -0.007785826317328924, 7: -0.012250983213056772, 8: 0.028288136395612062, 9: 0.009732627305648662, 10: -0.004914484376877558, 11: -0.0007686511283852137, 12: 0.006309678312853827, 13: -0.030436975866052096}
CPU times: user 2d 5h 58min 35s, sys: 58min 55s, total: 2d 6h 57min 31s
Wall time: 37min 39s


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])
/tmp/ipykernel_353353/2554929782.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred'] / cv


In [18]:
print(wPCC_asset_list)

{0: 0.03010742017410797, 1: 0.011425885666727932, 2: -0.016368366672512798, 3: 0.09981973136869705, 4: -0.04166187931922221, 5: -0.010135255686296042, 6: -0.007785826317328924, 7: -0.012250983213056772, 8: 0.028288136395612062, 9: 0.009732627305648662, 10: -0.004914484376877558, 11: -0.0007686511283852137, 12: 0.006309678312853827, 13: -0.030436975866052096}


In [19]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'avg'
# missing data
timestamp_fill = True
method = 'drop'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029831 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1234155, number of used features: 56
[LightGBM] [Info] Start training from score 0.000018
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047944 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 1230466, number of used features: 56
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


...

/tmp/ipykernel_353353/2554929782.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[0].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: 0.038515175840893895, 1: 0.04665673200041367, 2: -0.014464462296010644, 3: 0.09912634708837599, 4: -0.04335098973986725, 5: -0.008627295327630286, 6: -0.0004843933360413618, 7: 0.00025403367960254327, 8: 0.028939628345748137, 9: 0.012043934375957714, 10: -0.005678201511800155, 11: -0.0018388621622448841, 12: 0.004462183798908829, 13: -0.029541211470266646}
CPU times: user 2d 5h 15min 4s, sys: 58min 40s, total: 2d 6h 13min 45s
Wall time: 37min 9s


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])
/tmp/ipykernel_353353/2554929782.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred'] / cv


In [20]:
print(wPCC_asset_list)

{0: 0.038515175840893895, 1: 0.04665673200041367, 2: -0.014464462296010644, 3: 0.09912634708837599, 4: -0.04335098973986725, 5: -0.008627295327630286, 6: -0.0004843933360413618, 7: 0.00025403367960254327, 8: 0.028939628345748137, 9: 0.012043934375957714, 10: -0.005678201511800155, 11: -0.0018388621622448841, 12: 0.004462183798908829, 13: -0.029541211470266646}


In [21]:
%%time
# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'avg'
# missing data
timestamp_fill = True
method = 'linear'
weight = weight
df = df

wPCC_asset_list = main_lgb(df, cv, plength, ptrain, pgap, cv_method, timestamp_fill, method, weight)
print(wPCC_asset_list)
ws.append(wPCC_asset_list)

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010190 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6483
[LightGBM] [Info] Number of data points in the train set: 1251925, number of used features: 50
[LightGBM] [Info] Start training from score 0.000016
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020297 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6713
[LightGBM] [Info] Number of data points in the train set: 1256790, number of used features: 56
[LightGBM] [Info] Start training from score -0.000006
[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Volume']*train_data['VWAP']


...

/tmp/ipykernel_353353/2554929782.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = lgbs[0].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])


[LightGBM] [Warning] boosting is set with boost=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
{0: -0.012786174444071517, 1: -0.029236537368115174, 2: 0.02311135447249444, 3: -0.01911831052437244, 4: 0.013747679308018092, 5: -0.007232865762806147, 6: 0.00799382978775885, 7: -0.07916504268641271, 8: 0.02245215100311254, 9: 0.010028411408539578, 10: -0.008050942778692956, 11: 0.0018130627136209746, 12: -0.006871472333434657, 13: -0.040464469242493764}
CPU times: user 1d 19h 13min 3s, sys: 50min 33s, total: 1d 20h 3min 36s
Wall time: 30min 12s


/tmp/ipykernel_353353/2554929782.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred']+lgbs[i].lgr.predict(test_panel[feacols])
/tmp/ipykernel_353353/2554929782.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = test['pred'] / cv


In [22]:
print(wPCC_asset_list)

{0: -0.012786174444071517, 1: -0.029236537368115174, 2: 0.02311135447249444, 3: -0.01911831052437244, 4: 0.013747679308018092, 5: -0.007232865762806147, 6: 0.00799382978775885, 7: -0.07916504268641271, 8: 0.02245215100311254, 9: 0.010028411408539578, 10: -0.008050942778692956, 11: 0.0018130627136209746, 12: -0.006871472333434657, 13: -0.040464469242493764}


In [23]:
# w1 = {0: 0.09580684239594334, 1: 0.07421362452636102, 2: -0.022736489828096475, 3: 0.08872136242994266, 4: -0.03247886884977015, 5: -0.016533942431453685, 6: 0.007113471881949378, 7: -0.008941098895071694, 8: 0.02702699458657678, 9: 0.005687668528002063, 10: -0.008965269566128465, 11: 0.001989980880390933, 12: -0.00019171044602413523, 13: -0.025728794618873757}

# w2 = {0: 0.051716699118976424, 1: 0.006607246908253823, 2: 0.0011846759521534614, 3: 0.06653567533503459, 4: -0.039874209489066455, 5: -0.01303994045825557, 6: 0.014528910791308277, 7: 0.0038197475261504284, 8: 0.026226579986834715, 9: 0.00570904357543322, 10: -0.0040398207400765464, 11: -0.009743933852023429, 12: -0.0006029006768724578, 13: -0.026370359697096148}

# w3 = {0: 0.09595687620488583, 1: 0.0741093268989815, 2: -0.02273672988321399, 3: 0.08865574117747035, 4: -0.03247773183982738, 5: -0.01636650874552897, 6: 0.007126843025368912, 7: -0.009645886483590991, 8: 0.02702697107218858, 9: 0.005681454554419513, 10: -0.008964788625278436, 11: 0.002007550001818251, 12: -0.00019579330893962826, 13: -0.025723198582410855}

# w4 = {0: 0.012581992490019392, 1: 0.06941913452327876, 2: 0.01180237633077455, 3: -0.0024471790456864284, 4: 0.019180619031830225, 5: -0.00992623184294263, 6: -0.007171438197714991, 7: 0.029004429035334796, 8: 0.021402111639919494, 9: 0.012175855777688562, 10: -0.0026668981441261563, 11: 0.004526478312423389, 12: -0.006585184316443706, 13: 0.015898040866626707}

# w5 = {0: 0.03845520298547129, 1: 0.046652724053480164, 2: -0.014472851806897715, 3: 0.09905120978317249, 4: -0.04335064830863362, 5: -0.008574866917624333, 6: -0.0004610267300474252, 7: -0.0002509026224519182, 8: 0.029069001258359133, 9: 0.012129607001973575, 10: -0.005672114909042933, 11: -0.0019319571557174571, 12: 0.004463835487348649, 13: -0.02952084426687278}

# w6 = {0: 0.029679684275104045, 1: 0.0114456882014381, 2: -0.01684829960041441, 3: 0.09983411935511306, 4: -0.04166654222053501, 5: -0.010123741413109167, 6: -0.007796999466823302, 7: -0.012784461135539401, 8: 0.02864972154131925, 9: 0.009713555389140502, 10: -0.004906726012906564, 11: -0.0008066183372263516, 12: 0.006307383674165311, 13: -0.03041927915414989}

# w7 = {0: 0.03852497372408808, 1: 0.046646470271413915, 2: -0.01457539919652426, 3: 0.0990397512563344, 4: -0.043351210984529744, 5: -0.008581435095914027, 6: -0.00052123592583022, 7: -0.0004932407132410919, 8: 0.028978074686519677, 9: 0.012055158017300178, 10: -0.005667546887759213, 11: -0.001851256069574203, 12: 0.004459209704706342, 13: -0.029514022509826366}

# w8 = {0: -0.01278559040696362, 1: -0.029233847041174738, 2: 0.023295399356268173, 3: -0.019118314115085747, 4: 0.013747719790004888, 5: -0.007233528599890783, 6: 0.007993395603492513, 7: -0.07922378527310116, 8: 0.02245663574522049, 9: 0.010028338807072917, 10: -0.00805097668636171, 11: 0.001813862224086918, 12: -0.006875302398731842, 13: -0.04046449783275983}

# ws = [w1, w2, w3, w4, w5, w6, w7, w8]

cors = [sum(list(w.values())) for w in ws]

In [24]:
cors.index(max(cors)) + 1, max(cors)

(3, 0.18520609271333785)

In [25]:
print(cors)

[0.18513874369575883, 0.08232233294401758, 0.18520609271333785, 0.16708007351732707, 0.12584477451460593, 0.061361056643915904, 0.12601261928603957, -0.1237793264468549]


In [ ]:
# 第一个最好

[{0: 0.09620184390181802,
  1: 0.07421432838986834,
  2: -0.022736489828096573,
  3: 0.08900626786309172,
  4: -0.032479740518214596,
  5: -0.01638057137218082,
  6: 0.007126843025351401,
  7: -0.009643009292618939,
  8: 0.02706552708416497,
  9: 0.005636958868704742,
  10: -0.00895741222431711,
  11: 0.0019899808803909374,
  12: -0.00018311053213950192,
  13: -0.02572267255006376},
 {0: 0.05246783605879326,
  1: 0.0065545432996967955,
  2: 0.0007675356055733198,
  3: 0.0666621743367279,
  4: -0.03987825520331999,
  5: -0.013040664187379312,
  6: 0.014538169495402225,
  7: 0.0030907956883619104,
  8: 0.026245995796984316,
  9: 0.005703153301006097,
  10: -0.004039099904232205,
  11: -0.009771756984453806,
  12: -0.0006027205503427688,
  13: -0.02637537380880016},
 {0: 0.09578043773018241,
  1: 0.07412014797754697,
  2: -0.022926784700836145,
  3: 0.0888494837805019,
  4: -0.032477749829387344,
  5: -0.016397379445335685,
  6: 0.007122446690736907,
  7: -0.008901707087019234,
  8: 0.027

In [32]:
ws2 = pd.DataFrame.from_dict(ws).T
ws2['Asset_ID'] = ws2.index
ws2

,0,1,2,3,4,5,6,7,Asset_ID
0,0.096202,0.052468,0.095780,0.012582,0.038478,0.030107,0.038515,-0.012786,0
1,0.074214,0.006555,0.074120,0.069419,0.046663,0.011426,0.046657,-0.029237,1
2,-0.022736,0.000768,-0.022927,0.011817,-0.014675,-0.016368,-0.014464,0.023111,2
3,0.089006,0.066662,0.088849,-0.002447,0.099062,0.099820,0.099126,-0.019118,3
4,-0.032480,-0.039878,-0.032478,0.019181,-0.043352,-0.041662,-0.043351,0.013748,4
5,-0.016381,-0.013041,-0.016397,-0.009926,-0.008624,-0.010135,-0.008627,-0.007233,5
6,0.007127,0.014538,0.007122,-0.007171,-0.000479,-0.007786,-0.000484,0.007994,6
7,-0.009643,0.003091,-0.008902,0.028875,0.000168,-0.012251,0.000254,-0.079165,7
8,0.027066,0.026246,0.027232,0.021402,0.029096,0.028288,0.028940,0.022452,8
9,0.005637,0.005703,0.005680,0.012176,0.012089,0.009733,0.012044,0.010028,9


In [37]:
ws3 = pd.merge(ws2, weight, how='inner', left_on=['Asset_ID'], right_on=['Asset_ID'])
for i in range(0, 8):
    ws3[i] = ws3[i]/ws3['Weight']
ws3[['Asset_ID', 'Asset_Name', 'Weight']+list(range(0, 8))]

,Asset_ID,Asset_Name,Weight,0,1,2,3,4,5,6,7
0,0,Binance Coin,4.304065,0.022351,0.012190,0.022253,0.002923,0.008940,0.006995,0.008949,-0.002971
1,1,Bitcoin,6.779922,0.010946,0.000967,0.010932,0.010239,0.006883,0.001685,0.006882,-0.004312
2,2,Bitcoin Cash,2.397895,-0.009482,0.000320,-0.009561,0.004928,-0.006120,-0.006826,-0.006032,0.009638
3,3,Cardano,4.406719,0.020198,0.015127,0.020162,-0.000555,0.022480,0.022652,0.022494,-0.004338
4,4,Dogecoin,3.555348,-0.009135,-0.011216,-0.009135,0.005395,-0.012193,-0.011718,-0.012193,0.003867
5,5,EOS.IO,1.386294,-0.011816,-0.009407,-0.011828,-0.007160,-0.006221,-0.007311,-0.006223,-0.005217
6,6,Ethereum,5.894403,0.001209,0.002466,0.001208,-0.001217,-0.000081,-0.001321,-0.000082,0.001356
7,7,Ethereum Classic,2.079442,-0.004637,0.001486,-0.004281,0.013886,0.000081,-0.005891,0.000122,-0.038070
8,8,IOTA,1.098612,0.024636,0.023890,0.024787,0.019481,0.026484,0.025749,0.026342,0.020437
9,9,Litecoin,2.397895,0.002351,0.002378,0.002369,0.005078,0.005042,0.004059,0.005023,0.004182


In [38]:
ws3['Weight'].sum()

40.879847092769616